In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Create the large multiclass network
model = Network('MyNetwork')

In [3]:
# Block 1: Network nodes
node = np.empty(6, dtype=object)
node[0] = Source(model, 'Source')
node[1] = Queue(model, 'Queue1', SchedStrategy.FCFS)  # WebServer
node[2] = Queue(model, 'Queue2', SchedStrategy.FCFS)  # Storage1
node[3] = Queue(model, 'Queue3', SchedStrategy.PS)    # Storage2
node[4] = Queue(model, 'Queue4', SchedStrategy.FCFS)  # Storage3
node[5] = Sink(model, 'Sink')

In [4]:
# Block 2: Job classes with different characteristics
jobclass = np.empty(3, dtype=object)
jobclass[0] = OpenClass(model, 'Class1', 0)  # High-priority requests
jobclass[1] = OpenClass(model, 'Class2', 0)  # Medium-priority requests
jobclass[2] = OpenClass(model, 'Class3', 0)  # Low-priority requests

In [ ]:
# Arrival processes - different arrival rates per class
node[0].set_arrival(jobclass[0], Exp.fit_mean(5.000000))  # Class1: mean inter-arrival = 5.0 (rate = 0.2)
node[0].set_arrival(jobclass[1], Exp.fit_mean(8.000000))  # Class2: mean inter-arrival = 8.0 (rate = 0.125)
node[0].set_arrival(jobclass[2], Exp.fit_mean(7.000000))  # Class3: mean inter-arrival = 7.0 (rate = 0.143)

In [ ]:
# Service processes at WebServer (Queue1) - front-end processing
node[1].set_service(jobclass[0], Exp.fit_mean(0.300000))  # Class1: fast processing
node[1].set_service(jobclass[1], Exp.fit_mean(0.500000))  # Class2: medium processing
node[1].set_service(jobclass[2], Exp.fit_mean(0.600000))  # Class3: slower processing

In [ ]:
# Service processes at Storage1 (Queue2) - database access
node[2].set_service(jobclass[0], Exp.fit_mean(1.100000))  # Class1: complex queries
node[2].set_service(jobclass[1], Exp.fit_mean(1.300000))  # Class2: medium queries
node[2].set_service(jobclass[2], Exp.fit_mean(1.500000))  # Class3: simple queries

In [ ]:
# Service processes at Storage2 (Queue3) - file system access with PS
node[3].set_service(jobclass[0], Exp.fit_mean(2.000000))  # Class1: large files
node[3].set_service(jobclass[1], Exp.fit_mean(2.100000))  # Class2: medium files
node[3].set_service(jobclass[2], Exp.fit_mean(1.900000))  # Class3: small files

In [ ]:
# Service processes at Storage3 (Queue4) - cache access
node[4].set_service(jobclass[0], Exp.fit_mean(1.500000))  # Class1: cache miss handling
node[4].set_service(jobclass[1], Exp.fit_mean(0.900000))  # Class2: cache hits
node[4].set_service(jobclass[2], Exp.fit_mean(2.300000))  # Class3: cache updates

In [10]:
# Block 3: Complex routing topology with feedback loops
P = model.init_routing_matrix()

# Class1 routing
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)        # Source → WebServer
P.set(jobclass[0], jobclass[0], node[1], node[2], 0.25)       # WebServer → Storage1 (25%)
P.set(jobclass[0], jobclass[0], node[1], node[3], 0.25)       # WebServer → Storage2 (25%)
P.set(jobclass[0], jobclass[0], node[1], node[4], 0.25)       # WebServer → Storage3 (25%)
P.set(jobclass[0], jobclass[0], node[1], node[5], 0.25)       # WebServer → Sink (25%)
P.set(jobclass[0], jobclass[0], node[2], node[1], 1.0)        # Storage1 → WebServer (feedback)
P.set(jobclass[0], jobclass[0], node[3], node[1], 1.0)        # Storage2 → WebServer (feedback)
P.set(jobclass[0], jobclass[0], node[4], node[1], 1.0)        # Storage3 → WebServer (feedback)

# Class2 routing
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)        # Source → WebServer
P.set(jobclass[1], jobclass[1], node[1], node[2], 0.25)       # WebServer → Storage1 (25%)
P.set(jobclass[1], jobclass[1], node[1], node[3], 0.25)       # WebServer → Storage2 (25%)
P.set(jobclass[1], jobclass[1], node[1], node[4], 0.25)       # WebServer → Storage3 (25%)
P.set(jobclass[1], jobclass[1], node[1], node[5], 0.25)       # WebServer → Sink (25%)
P.set(jobclass[1], jobclass[1], node[2], node[1], 1.0)        # Storage1 → WebServer (feedback)
P.set(jobclass[1], jobclass[1], node[3], node[1], 1.0)        # Storage2 → WebServer (feedback)
P.set(jobclass[1], jobclass[1], node[4], node[1], 1.0)        # Storage3 → WebServer (feedback)

# Class3 routing
P.set(jobclass[2], jobclass[2], node[0], node[1], 1.0)        # Source → WebServer
P.set(jobclass[2], jobclass[2], node[1], node[2], 0.25)       # WebServer → Storage1 (25%)
P.set(jobclass[2], jobclass[2], node[1], node[3], 0.25)       # WebServer → Storage2 (25%)
P.set(jobclass[2], jobclass[2], node[1], node[4], 0.25)       # WebServer → Storage3 (25%)
P.set(jobclass[2], jobclass[2], node[1], node[5], 0.25)       # WebServer → Sink (25%)
P.set(jobclass[2], jobclass[2], node[2], node[1], 1.0)        # Storage1 → WebServer (feedback)
P.set(jobclass[2], jobclass[2], node[3], node[1], 1.0)        # Storage2 → WebServer (feedback)
P.set(jobclass[2], jobclass[2], node[4], node[1], 1.0)        # Storage3 → WebServer (feedback)

model.link(P)

In [11]:
# Configure solver options for the large network
options = lineDefaults()  # Call the function to get SolverOptions object
options['seed'] = 23000
options['samples'] = int(1e6)  # High sample count for accuracy
options['cutoff'] = 1         # Population cutoff for finite state approximation

In [ ]:
# Aligned with JAR test scenarios for oqn_fourqueues
# JAR tests: CTMC(seed=23000, cutoff=1), MVA(seed=23000), MAM(seed=23000), JMT(seed=23000, samples=1000000)

solver = np.array([], dtype=object)

# CTMC with seed=23000, cutoff=1 (matches JAR)
solver = np.append(solver, CTMC(model, seed=23000, cutoff=1))

# MVA with seed=23000 (matches JAR)
solver = np.append(solver, MVA(model, seed=23000))

# MAM (analytical solver - no seed needed)
solver = np.append(solver, MAM(model))

# JMT with seed=23000, samples=1000000 (matches JAR)
solver = np.append(solver, JMT(model, seed=23000, samples=1000000))

# DES with seed=23000, samples=1000000 (matches MATLAB)
solver = np.append(solver, DES(model, seed=23000, samples=1000000))

In [ ]:
# Execute all solvers and collect results
AvgTable = np.empty(len(solver), dtype=object)

for s in range(len(solver)):
    AvgTable[s] = solver[s].avg_table()